In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf() \
    .setAppName('SparkApp') \
    .setMaster('spark://spark:7077') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .set("spark.sql.shuffle.partitions", "10")
 

sc = SparkContext.getOrCreate(conf=conf)

from pyspark.sql import SQLContext
# Créer un SQLContext pour les opérations SQL
sql_context = SQLContext(sc)

minio_ip_address = "minio"

In [ ]:
%pip install ipyleaflet

In [ ]:
from ipyleaflet import Map, basemaps, Marker, Icon, Popup
from ipywidgets import HTML
import time

icon_url = "./avion_icon.png"
icon = Icon(icon_url=icon_url, icon_size=[40, 40])  # Taille ajustable

center = [45.3288, 1.0]

m = Map(center=center, zoom=5)

message2 = HTML()
message2.value = "Hello <b>World</b>"
message2.placeholder = "Some HTML"
message2.description = "Some HTML"

marker = Marker(location=center, icon=icon, draggable=False)
marker.popup = message2

m.add(marker)

display(m)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, avg
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, FloatType

# Kafka configuration
kafka_broker = "kafka1:9092"
kafka_topic = "planes"

# Define schema for the Kafka message
schema = StructType([
    StructField("icao24", StringType(), True),
    StructField("origin_country", StringType(), True),
    StructField("velocity", FloatType(), True),
    StructField("timestamp", StringType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True)
])

# Read raw data from Kafka
raw_stream = sql_context.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

df_parsed = raw_stream.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# # Output rolling average to the console
query = df_parsed.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

In [ ]:
query.stop()